# RSNA-2025 EDA / Baseline (Colab)

このノートブックは Colab 実行を前提とし、GCS バケット `rsna2025-prod` から `train.csv` と `train_localizers.csv` を読み込み、EDA → ベースライン学習 → 参考用の推論関数雛形までをまとめます。

- 実行順序: セットアップ → データロード → EDA 可視化 → ベースライン学習/検証
- データ配置: GCS（`gs://rsna2025-prod/`）
- 参考リポジトリ: `https://github.com/Kohei-Arita/RSNA-2025.git`（必要に応じてclone）

注意: Kaggle の `/kaggle/input/...` 参照コードは Colab では使用しません。GCS から直接読み込みます。


In [ ]:
# 0) セットアップ（Colab）: 依存導入・GCS認証・GitHub clone
import os
import sys
import subprocess
from pathlib import Path

IN_COLAB = 'google.colab' in sys.modules
print('IN_COLAB =', IN_COLAB)

if IN_COLAB:
    # pip を Python から実行（ノートブックマジックを使わない）
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', '-U', 'pip'], check=True)
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q',
                    'pandas', 'polars', 'seaborn', 'scikit-learn', 'matplotlib', 'gcsfs', 'fsspec', 'pydicom'], check=True)

    # GCP 認証（ADC）。対話UIが出ます
    from google.colab import auth  # type: ignore
    auth.authenticate_user()

    # 作業ディレクトリを /content に設定
    os.chdir('/content')

    # GitHub から本リポジトリを取得
    REPO_URL = 'https://github.com/Kohei-Arita/RSNA-2025.git'
    REPO_DIR = Path('/content/RSNA-2025')
    if not REPO_DIR.exists():
        subprocess.run(['git', 'clone', REPO_URL], check=True)
    os.chdir('/content/RSNA-2025')

    # Colab ランタイムにリポジトリの src を追加（任意）
    sys.path.insert(0, str(Path.cwd() / 'src'))

# GCS バケット設定
GCS_BUCKET = 'rsna2025-prod'
GCS_BASE = f'gs://{GCS_BUCKET}'
print('GCS_BASE =', GCS_BASE)



In [ ]:
# 1) データ読込（GCS / gcsfs 経由）
import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

sns.set_theme(style='whitegrid')
pd.options.display.max_columns = 120

SEED = 130

train_uri = f'{GCS_BASE}/train.csv'
trainloc_uri = f'{GCS_BASE}/train_localizers.csv'

# ColabのADCを gcsfs が利用（storage_options={'token': 'cloud'}）
train = pd.read_csv(train_uri, storage_options={'token': 'cloud'})
train_localizers = pd.read_csv(trainloc_uri, storage_options={'token': 'cloud'})

print(f"Number of training series: {train.shape[0]}")
print(f"Number of localization rows: {train_localizers.shape[0]}")

display(train.head())
display(train_localizers.head())



In [ ]:
# 2) EDA 可視化
# 年齢を数値化（"xx - yy" 形式の先頭、または数字抽出）
df_age_str = train['PatientAge'].astype(str)
age_first = df_age_str.str.split(' - ').str[0]
age_vals = pd.to_numeric(age_first.str.extract(r'([0-9]+(?:\.[0-9]+)?)')[0], errors='coerce')

plt.figure(figsize=(8,4))
plt.hist(age_vals.dropna(), bins=20, edgecolor='k')
plt.title('Patient Age Distribution')
plt.xlabel('Age')
plt.ylabel('Count')
plt.show()

plt.figure(figsize=(6,4))
sns.countplot(data=train, x='PatientSex', palette='pastel')
plt.title('Patient Sex Distribution')
plt.xlabel('Sex')
plt.ylabel('Count')
plt.show()

# 箱ひげ（欠損を除去）
tmp = train.copy()
tmp['age_num'] = age_vals
plt.figure(figsize=(8,4))
sns.boxplot(x='Aneurysm Present', y='age_num', data=tmp)
plt.xticks([0,1], ['Absent','Present'])
plt.title('Age vs. Aneurysm Presence')
plt.ylabel('Age')
plt.show()

plt.figure(figsize=(6,4))
sns.countplot(data=train, x='Modality', palette='Set2')
plt.title('Imaging Modality Counts')
plt.xlabel('Modality')
plt.ylabel('Count')
plt.show()

label_cols = [
    'Left Infraclinoid Internal Carotid Artery','Right Infraclinoid Internal Carotid Artery',
    'Left Supraclinoid Internal Carotid Artery','Right Supraclinoid Internal Carotid Artery',
    'Left Middle Cerebral Artery','Right Middle Cerebral Artery','Anterior Communicating Artery',
    'Left Anterior Cerebral Artery','Right Anterior Cerebral Artery',
    'Left Posterior Communicating Artery','Right Posterior Communicating Artery',
    'Basilar Tip','Other Posterior Circulation'
]

# CSV の実カラムと突き合わせ（存在する列のみ使う）
existing_labels = [c for c in label_cols if c in train.columns]
if len(existing_labels) < len(label_cols):
    print('Warning: 一部のラベル列が見つかりませんでした。検出:', len(existing_labels))

prevalences = train[existing_labels].mean().sort_values(ascending=False)
plt.figure(figsize=(10,6))
sns.barplot(x=prevalences.values, y=prevalences.index, palette='coolwarm')
plt.title('Aneurysm Prevalence by Vascular Location')
plt.xlabel('Prevalence')
plt.show()

# 相関ヒートマップ
cols_for_corr = existing_labels + (['Aneurysm Present'] if 'Aneurysm Present' in train.columns else [])
if len(cols_for_corr) >= 2:
    plt.figure(figsize=(12,10))
    cor_mat = train[cols_for_corr].corr(numeric_only=True)
    sns.heatmap(cor_mat, annot=False, cmap='vlag')
    plt.title('Correlation Matrix of Label Columns')
    plt.show()



In [ ]:
# 3) ベースライン前処理・学習・評価（GBM）
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier

# 特徴量: 年齢・性別（Male=1）・モダリティone-hot
x_age = age_vals.fillna(age_vals.median())
X = pd.DataFrame({
    'age': x_age,
    'sex': (train['PatientSex'] == 'Male').astype(int)
})
mod_dummies = pd.get_dummies(train['Modality'], prefix='mod')
X = pd.concat([X, mod_dummies], axis=1)

# 目的変数
if train['Aneurysm Present'].dtype != np.int64 and train['Aneurysm Present'].dtype != np.int32:
    y = train['Aneurysm Present'].astype(int)
else:
    y = train['Aneurysm Present']

print(f"Feature matrix shape: {X.shape}")

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=SEED
)

gbm = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=8,
    random_state=SEED
)

gbm.fit(X_train, y_train)
val_probs = gbm.predict_proba(X_val)[:, 1]
val_auc = roc_auc_score(y_val, val_probs)
print(f"GBM Validation AUC: {val_auc:.4f}")

# 後続で利用するために列名を保持
MOD_COLUMNS = list(mod_dummies.columns)



In [ ]:
# 4) 参考: 簡易予測関数とサブミッション雛形（ローカル検証用）
# Kaggle のサービングAPIは Colab では起動しません。ここでは簡易に presence を全ラベルに複写した雛形を作ります。
ID_COL = 'SeriesInstanceUID'
LABEL_COLS = [
    'Left Infraclinoid Internal Carotid Artery','Right Infraclinoid Internal Carotid Artery',
    'Left Supraclinoid Internal Carotid Artery','Right Supraclinoid Internal Carotid Artery',
    'Left Middle Cerebral Artery','Right Middle Cerebral Artery','Anterior Communicating Artery',
    'Left Anterior Cerebral Artery','Right Anterior Cerebral Artery',
    'Left Posterior Communicating Artery','Right Posterior Communicating Artery',
    'Basilar Tip','Other Posterior Circulation','Aneurysm Present'
]

# フォールバック平均
available_label_cols = [c for c in LABEL_COLS if c in train.columns]
means = train[available_label_cols].mean(numeric_only=True).to_dict()
train_idx = train.set_index(ID_COL)

def _build_feature_row(row):
    age_val = pd.to_numeric(str(row['PatientAge']).split(' - ')[0], errors='coerce')
    if pd.isna(age_val):
        age_val = age_vals.median()
    sex_val = 1 if row['PatientSex'] == 'Male' else 0
    feats = {'age': age_val, 'sex': sex_val}
    for m in MOD_COLUMNS:
        feats[m] = 1 if m == f"mod_{row['Modality']}" else 0
    return pd.DataFrame([feats])

# デモ: 学習データの先頭N件に対して presence を推定し、全ラベルに複写
def build_submission_preview(n=10):
    rows = []
    for sid, row in train.head(n).set_index(ID_COL).iterrows():
        feat_df = _build_feature_row(row)
        prob = float(gbm.predict_proba(feat_df)[:, 1][0])
        out = [sid] + [prob for _ in LABEL_COLS]
        rows.append(out)
    df = pl.DataFrame(rows, schema=[ID_COL] + LABEL_COLS)
    return df

submission_preview = build_submission_preview(10)
submission_preview.head(3)


In [ ]:
# 0) セットアップ（Colab）: 依存導入・GCS認証・GitHub clone
import os, sys, subprocess
from pathlib import Path

IN_COLAB = 'google.colab' in sys.modules
print('IN_COLAB =', IN_COLAB)

if IN_COLAB:
    # pip を Python から実行（ノートブックマジックを使わない）
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', '-U', 'pip'], check=True)
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q',
                    'pandas', 'polars', 'seaborn', 'scikit-learn', 'matplotlib',
                    'gcsfs', 'fsspec', 'pydicom', 'nibabel',
                    'opencv-python-headless',
                    'pylibjpeg[all]'], check=True)  # DICOMの可逆/非可逆圧縮に備える

    # GCP 認証（ADC）。対話UIが出ます
    from google.colab import auth  # type: ignore
    auth.authenticate_user()

    # 作業ディレクトリ
    os.chdir('/content')

    # GitHub から本リポジトリを取得（必要に応じて）
    REPO_URL = 'https://github.com/Kohei-Arita/RSNA-2025.git'
    REPO_DIR = Path('/content/RSNA-2025')
    if not REPO_DIR.exists():
        subprocess.run(['git', 'clone', REPO_URL], check=True)
    os.chdir('/content/RSNA-2025')

    # Colab ランタイムにリポジトリの src を追加（任意）
    sys.path.insert(0, str(Path.cwd() / 'src'))

# GCS バケット設定
GCS_BUCKET = 'rsna2025-prod'
GCS_BASE = f'gs://{GCS_BUCKET}'
print('GCS_BASE =', GCS_BASE)

In [ ]:
# 2.5) 画像EDA（GCS版：他者コードの内容をGCSに適合）
# 参照情報は Kaggle ノートの記述を維持し、入出力のみ GCS に切替

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings, random, io, tempfile, math

import fsspec
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import nibabel as nib

warnings.simplefilter(action='ignore', category=Warning)

# --- GCS FS 準備 ---
fs = fsspec.filesystem("gcs", token="cloud")  # ColabのADCを使用

# --- (Kaggle相当) test.csv のヘッド確認：存在すれば読む ---
# Kaggleコードでは ../input/.../kaggle_evaluation/test.csv を参照していたため、
# あなたのバケットの kaggle_evaluation/test.csv を優先的に探します（無ければスキップ）
test_uri = f"{GCS_BASE}/kaggle_evaluation/test.csv"
try:
    if fs.exists(test_uri):
        with fs.open(test_uri, 'rb') as f:
            test = pd.read_csv(f)
        print("test: No more data to show (head)")
        display(test.head())
    else:
        print(f"[Info] {test_uri} が見つからないため test.csv のプレビューはスキップします。")
except Exception as e:
    print(f"[Warn] test.csv の読み込みをスキップしました: {e}")

# --- train_localizers: 既に上流で train_localizers を読み込んでいる前提 ---
# Kaggleノートの説明文相当は省略、データ構造は上流のDataFrameで満たしています
print("train_localizers head (from GCS):")
display(train_localizers.head(2))

# --- Modality の分布（他者コードの value_counts 相当）
print("train['Modality'].value_counts()")
print(train['Modality'].value_counts())

# --- 数値列の相関（他者コードの heatmap 相当）
numerical_cols = [
    'PatientAge',
    'Left Infraclinoid Internal Carotid Artery',
    'Right Infraclinoid Internal Carotid Artery',
    'Left Supraclinoid Internal Carotid Artery',
    'Right Supraclinoid Internal Carotid Artery',
    'Left Middle Cerebral Artery',
    'Right Middle Cerebral Artery',
    'Anterior Communicating Artery',
    'Left Anterior Cerebral Artery',
    'Right Anterior Cerebral Artery',
    'Left Posterior Communicating Artery',
    'Right Posterior Communicating Artery',
    'Basilar Tip',
    'Other Posterior Circulation',
    'Aneurysm Present'
]

plt.figure(figsize=(16, 8))
corr_mat = train[numerical_cols].corr(numeric_only=True)
sns.heatmap(corr_mat, annot=True, cmap='summer')
plt.title("Correlation Between Numerical Features")
plt.show()

# --- DICOM シリーズ（series/**.dcm）の列挙とサンプル表示 ---
# Kaggleコード: glob('../input/.../series/**/*.dcm') → GCS: fs.find('gs://.../series')
print("Listing DICOMs under GCS ... (this may take some time for the first call)")
all_series_paths = fs.find(f"{GCS_BASE}/series")
train_images = [p for p in all_series_paths if p.lower().endswith('.dcm')]
print("Total number of images: ", len(train_images))

# 代表1枚のメタ情報＆ピクセル配列タイプの確認（他者コードの print(ds), type/im.dtype/im.shape 相当）
def read_dicom_from_gcs(path: str) -> pydicom.dataset.FileDataset:
    # fsspec file-like をそのまま pydicom に渡せます
    with fs.open(path, 'rb') as f:
        ds = pydicom.dcmread(f, force=True)  # 圧縮対応は pylibjpeg[all] のインストールで担保
        try:
            # ビューワ向けに VOI LUT を適用したピクセルを得る（あれば）
            arr = ds.pixel_array
            try:
                arr = apply_voi_lut(arr, ds)
            except Exception:
                pass
        except Exception:
            arr = None
    return ds

if len(train_images) > 0:
    sample_path = train_images[min(1, len(train_images)-1)]
    ds = read_dicom_from_gcs(sample_path)
    print(ds)  # ファイルタイプなどの概要
    try:
        im = ds.pixel_array
        print(type(im))
        print(im.dtype)
        print(im.shape)
    except Exception as e:
        print(f"[Warn] pixel_array 取得に失敗: {e}")

    # 1枚表示（他者コード: pylab.imshow(im, cmap=...）相当）
    try:
        import matplotlib.pylab as pylab
        pylab.imshow(ds.pixel_array, cmap=pylab.cm.gist_gray)
        pylab.axis('on')
        plt.show()
    except Exception as e:
        print(f"[Warn] DICOM表示に失敗: {e}")

# グリッドで複数表示（他者コードの for ループ可視化に相当）
def show_random_dcms(paths, rows=4, cols=5):
    n = min(len(paths), rows*cols)
    if n == 0:
        print("[Info] 表示対象のDICOMがありません。")
        return
    sel = random.sample(paths, n) if len(paths) > n else paths[:n]
    fig = plt.figure(figsize=(15, 10))
    for i, p in enumerate(sel, 1):
        try:
            ds = read_dicom_from_gcs(p)
            ax = fig.add_subplot(rows, cols, i)
            ax.imshow(ds.pixel_array, cmap=plt.cm.bone)
            ax.set_xticks([]); ax.set_yticks([])
        except Exception as e:
            # 壊れたファイルがあっても全体は止めない
            pass
    plt.tight_layout()
    plt.show()

show_random_dcms(train_images, rows=4, cols=5)

# --- NIfTI セグメンテーション（segmentations/*/*.nii）の列挙＆表示 ---
# Kaggleコードのパターン: '../input/.../segmentations/*/*'
print("Listing NIfTI masks under GCS ...")
all_seg_paths = fs.find(f"{GCS_BASE}/segmentations")  # 再帰列挙
mrscans = sorted([p for p in all_seg_paths if p.lower().endswith('.nii')])
print("num of segmentation mask", len(mrscans))

def load_nifti_from_gcs(path: str):
    # nib は file-like 直接は非対応のことがあるため、一時ファイルに流す
    with fs.open(path, 'rb') as fsrc, tempfile.NamedTemporaryFile(suffix=".nii", delete=False) as fdst:
        fdst.write(fsrc.read())
        tmp_path = fdst.name
    img = nib.load(tmp_path).get_fdata()
    return img, tmp_path  # tmp_pathは後で削除してもOK

# 1例：shape 確認と1スライス表示（Kaggleの例：imshow(img[:,:,10])）
if len(mrscans) > 0:
    try:
        img, tmp = load_nifti_from_gcs(mrscans[0])
        print("NIfTI shape:", img.shape)
        plt.imshow(img[:, :, min(10, img.shape[2]-1)])
        plt.title("Example NIfTI slice")
        plt.axis('off')
        plt.show()
        try:
            os.remove(tmp)
        except Exception:
            pass
    except Exception as e:
        print(f"[Warn] NIfTI 読み込み表示に失敗: {e}")

# 複数スライスをグリッドで可視化（他者コードの multi_dim_plot を踏襲）
def multi_dim_plot(multi_dim_array, id, num_slices=25):  # ※元コードのバリエーションに合わせて 25
    fig = plt.figure(figsize=(30, 30))
    plt.title(f'Plotting first {num_slices} slices of {id}', fontdict={'fontsize': 20})
    plt.yticks([]); plt.xticks([])
    xy = int(math.sqrt(num_slices))
    for i in range(num_slices):
        ax = fig.add_subplot(xy, xy, i + 1)
        ax.imshow(multi_dim_array[..., :num_slices][..., i])
        ax.axis("off")
    plt.show()

# ランダムに2例を可視化（元ノートの振る舞いを踏襲）
if len(mrscans) > 0:
    k = min(2, len(mrscans))
    for msk_path in random.sample(mrscans, k):
        try:
            m, tmp = load_nifti_from_gcs(msk_path)
            multi_dim_plot(m, id=msk_path.split('/')[-1], num_slices=25)
            try:
                os.remove(tmp)
            except Exception:
                pass
        except Exception as e:
            print(f"[Warn] NIfTI 可視化に失敗: {e}")
else:
    print("[Info] 表示対象の NIfTI マスクがありません。")